In [ ]:
import particle_tracking as pt

pt.track_particle_burns(r"F:\02092025\test6\test6_calibrated", 2000, 15000, stop=False)

In [ ]:
# reads data from a csv generated by particle_tracking.py
# Generates a dictionary called data_dict with keys as particle IDs and 'frames', 'xcoords', 'ycoords', and 'brightnesses' as sub-keys
# then trims particles with only one frame

import matplotlib.pyplot as plt
import numpy as np

#open csv file
with open(r"F:\02092025\test6\test6_calibrated_particle_tracking_results.csv") as f:
    data = f.read().split('\n')
    lines = []
    for line in data:
        lines.append(line.split(','))

#create data_dict
i = 1
data_dict = {}
while i < len(lines)-1:
    id = lines[i][0]
    frames = lines[i+1]
    xcoords = lines[i+2]
    ycoords = lines[i+3]
    brightnesses = lines[i+4]
    data_dict[id] = {}
    data_dict[id]['frames'] = []
    data_dict[id]['xcoords'] = []
    data_dict[id]['ycoords'] = []
    data_dict[id]['brightnesses'] = []
    for j in range(1, len(frames)):
        data_dict[id]['frames'].append(int(frames[j]))
        if xcoords[j] == '':
            data_dict[id]['xcoords'].append(None)
            data_dict[id]['ycoords'].append(None)
            data_dict[id]['brightnesses'].append(None)
        else:
            data_dict[id]['xcoords'].append(int(xcoords[j]))
            data_dict[id]['ycoords'].append(int(ycoords[j]))
            data_dict[id]['brightnesses'].append(int(float(brightnesses[j])))
    i += 5

#trim 1-frame particles
ids = list(data_dict.keys())
for id in ids:
    if len(data_dict[id]['frames']) == 1:
        data_dict.pop(id)

#find average starting y position, corresponding to laser position
startys = []
for id in data_dict.keys():
    starty = data_dict[id]['ycoords'][0]
    if starty < 400:
        print(id)
        print(data_dict[id]['frames'][0])
    startys.append(starty)
startys_mm = [y*0.021 for y in startys if y is not None]
mean_starty = np.mean(startys)
print(mean_starty)

#find y position of particles at their brightest point
brightestys = {}
for id in data_dict.keys():
    brightnesses = data_dict[id]['brightnesses']
    if all(b is None for b in brightnesses):
        continue
    max_brightness = max([b for b in brightnesses if b is not None])
    max_index = brightnesses.index(max_brightness)
    brightesty = data_dict[id]['ycoords'][max_index]
    if brightesty is not None:
        brightestys[id] = brightesty

#plot histogram of y positions at brightest point
#red margins indicate laser position (+- 8 pixels)
plt.figure(1)
hist = plt.hist(brightestys.values(), bins=50, color='blue', alpha=0.7)
plt.plot([mean_starty+8, mean_starty+8], [0, 800], color='red', linestyle='--')
plt.plot([mean_starty-8, mean_starty-8], [0, 800], color='red', linestyle='--')


max_brightnesses = {}
for id in data_dict.keys():
    brightnesses = data_dict[id]['brightnesses']
    if all(b is None for b in brightnesses):
        continue
    max_brightness = max([b for b in brightnesses if b is not None])
    max_brightnesses[id] = max_brightness
plt.figure(2)
hist = plt.hist(max_brightnesses.values(), bins=50, color='blue', alpha=0.7)

#trim particles that don't reach a certain maximum brightness
ids = list(data_dict.keys())
for id in ids:
    brightnesses = data_dict[id]['brightnesses']
    if all(b is None for b in brightnesses):
        continue
    max_brightness = max([b for b in brightnesses if b is not None])
    if max_brightness < 10000:
        data_dict.pop(id)

#trim particles that reach their brightest point inside the laser
ids = list(data_dict.keys())
for id in ids:
    if brightestys[id] > mean_starty - 15 and brightestys[id] < mean_starty + 15:
        data_dict.pop(id)

#calculate burn times for remaining particles
burn_times = []
fps = 15000
for id in data_dict.keys():
    burn_time = (data_dict[id]['frames'][-1] - data_dict[id]['frames'][0])/fps*1000
    burn_times.append(burn_time)
plt.figure(3)
hist = plt.hist(burn_times, bins=25, color='blue', alpha=0.7)
plt.xlabel('burn time (ms)')
plt.ylabel('count')
plt.title('burn times, trimmed at 10000 max brightness and 15 pix from laser at max brightness')
print(burn_times)
print(len(data_dict.keys()))
